In [7]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("Orders_DB.db")

In [46]:
# displaying DB tables
pd.read_sql("""
select name
from sqlite_master
where type = 'table';
""", conn)

,name
0,Users
1,Orders
2,Order_Items


### Initial Queries to Explore Table Structure
Always want to begin analysis/query writing in general with a full grasp of the tables.
Helpful to get a lay of the land - what are PK's and FK's, how to tables relate, what kind of table structure we have, etc.

In [47]:
# See all orders
pd.read_sql("""

select *
from users;

""", conn)

,User_ID,First_Name,Last_Name,SignUp_Date,Country
0,1,Mason,Webb,1/5/24,US
1,2,Alex,Grimes,2/11/24,US
2,3,Jordan,Klein,2/20/24,DE
3,4,Chris,Collins,3/1/24,GB
4,5,Daniel,Plainview,3/15/24,CA


"User_ID" is the primary key, can see how it links to the other tables after running the queries below.

In [48]:
# see all orders
pd.read_sql("""

select *
from orders;

""", conn)

,Order_ID,User_ID,Order_Date,Order_Total
0,101,1,3/1/25,120.50
1,102,2,3/5/25,58.99
2,103,1,3/12/25,200.00
3,104,3,3/12/25,89.54
4,105,4,3/15/25,250.00
5,106,5,3/20/25,19.99


"Order_ID" is PK here, FK from the user table is "User_ID".

In [15]:
# see all order items
pd.read_sql("""

select *
from order_items;

""", conn)

,Order_Item_ID,Order_ID,Product_Name,Quantity,Price
0,1,101,Pair Running Shoes,1,150.00
1,2,101,Pair Socks,2,10.25
2,3,102,Water Bottle,1,64.99
3,4,103,Protein Powder,3,25.50
4,5,104,Pair Running Shoes,1,150.00
5,6,104,Gym Shorts,1,50.00
6,7,104,Tank Top,1,30.45
7,8,105,Gym Bag,1,15.79
8,9,106,Foam Roller,2,43.56
9,10,107,Everyday Supplements,1,30.00


"Order_Item_ID" is the PK in this table. "Order_ID" is FK from the orders table.

### Getting Insights
Now that we have an understanding of the tables, we can start to aggreagte the data some and get some takeaways. 
Will be doing basic joins and functions in these examples due to limtations of a sample DB only having so many tables. In a real scenario there are often many hurdles with data quality and cleaning that you'd have to also handle here before moving to a BI tool such as Sigma or Tableau - this is meant to just show understanding on DB relationships as well as the ability to answer relevant business questions.

In [18]:
# orders with usernames, sorted by order date
pd.read_sql("""
select 
u.first_name
, u.last_name
, o.order_id
, o.order_date
from orders o
left join users u
  on u.user_id = o.user_id
order by o.order_date desc;
""", conn)

,First_Name,Last_Name,Order_ID,Order_Date
0,Alex,Grimes,102,3/5/25
1,Daniel,Plainview,106,3/20/25
2,Chris,Collins,105,3/15/25
3,Mason,Webb,103,3/12/25
4,Jordan,Klein,104,3/12/25
5,Mason,Webb,101,3/1/25


In [19]:
# Total order value per order (by user) and sorted by revenue
pd.read_sql("""

select 
u.First_Name
, u.Last_Name
, oi.Order_ID
, SUM(oi.Price) AS total_price
from orders o
left join users u
  on u.User_ID = o.User_ID
left join Order_Items oi
  on oi.Order_ID = o.Order_ID
group by u.First_Name, u.Last_Name, oi.Order_ID
order by total_price DESC;

""", conn)

,First_Name,Last_Name,Order_ID,total_price
0,Jordan,Klein,104,230.45
1,Mason,Webb,101,160.25
2,Alex,Grimes,102,64.99
3,Daniel,Plainview,106,43.56
4,Mason,Webb,103,25.50
5,Chris,Collins,105,15.79


In [24]:
# Most popular product by quantity
pd.read_sql("""
select
product_name
, sum(quantity) as total_ordered
from order_items
group by product_name
order by total_ordered desc;
""",conn)

,Product_Name,total_ordered
0,Protein Powder,3
1,Pair Socks,2
2,Pair Running Shoes,2
3,Foam Roller,2
4,Water Bottle,1
5,Tank Top,1
6,Gym Shorts,1
7,Gym Bag,1
8,Everyday Supplements,1


In [29]:
# users by total items ordered
pd.read_sql("""
select
u.first_name
, u.last_name
, sum(oi.quantity) as total_products_ordered
from orders o
left join users u
    on u.user_id = o.user_id
left join order_items oi
    on oi.order_id = o.order_id
group by u.first_name, u.last_name
order by total_products_ordered desc;
""", conn)

,First_Name,Last_Name,total_products_ordered
0,Mason,Webb,6
1,Jordan,Klein,3
2,Daniel,Plainview,2
3,Alex,Grimes,1
4,Chris,Collins,1


In [30]:
# users by total orders
pd.read_sql("""
select
u.first_name
, u.last_name
, count(distinct o.order_id) as total_orders_made
from orders o
left join users u
    on u.user_id = o.user_id
group by u.first_name, u.last_name
order by total_orders_made desc;

""",conn
)

,First_Name,Last_Name,total_orders_made
0,Mason,Webb,2
1,Alex,Grimes,1
2,Chris,Collins,1
3,Daniel,Plainview,1
4,Jordan,Klein,1


In [35]:
# total revenue by day
pd.read_sql("""
select
o.order_date
, sum(o.order_total) as total_revenue
from orders o
group by o.order_date
order by total_revenue desc;
""",conn
)

,Order_Date,total_revenue
0,3/12/25,289.54
1,3/15/25,250.00
2,3/1/25,120.50
3,3/5/25,58.99
4,3/20/25,19.99


In [36]:
# average revenue by day
pd.read_sql("""
select avg(order_total) as avg_order_total from orders;

""",conn
)

,avg_order_total
0,123.17


In [40]:
# time between orders per user
pd.read_sql("""
select
u.user_id
, u.first_name
, u.last_name
, o.order_date
, lead(o.order_date) over (partition by u.user_id order by o.order_date desc) as previous_order_date
from orders o
left join users u
    on u.user_id = o.user_id
order by u.user_id desc, o.order_date desc

""",conn
)

,User_ID,First_Name,Last_Name,Order_Date,previous_order_date
0,5,Daniel,Plainview,3/20/25,None
1,4,Chris,Collins,3/15/25,None
2,3,Jordan,Klein,3/12/25,None
3,2,Alex,Grimes,3/5/25,None
4,1,Mason,Webb,3/12/25,3/1/25
5,1,Mason,Webb,3/1/25,None


In [45]:
# orders by revenue and country
pd.read_sql("""
select
u.country
, sum(o.order_total) as total_order_price
, count(distinct o.order_id) as total_orders
, round(avg(o.order_total),2) as avg_order_price
from orders o
left join users u
    on u.user_id = o.user_id
group by u.country
order by total_order_price desc;

""",conn
)

,Country,total_order_price,total_orders,avg_order_price
0,US,379.49,3,126.50
1,GB,250.00,1,250.00
2,DE,89.54,1,89.54
3,CA,19.99,1,19.99
